In [34]:
import pandas as pd
from pathlib import Path

In [35]:
dataset_path = Path("df_clusterized.pkl")
df = pd.read_pickle(dataset_path)
df.head()

,cluster_label,emb,target
0,5.0,"[-0.19262028, 0.67613167, -0.06970257, 1.36109...",90000.0
1,5.0,"[-0.18461296, 0.79767823, -0.011102157, 1.4939...",90000.0
2,5.0,"[-0.20171373, 0.72063917, -0.051841334, 1.3009...",70000.0
3,12.0,"[-0.23105316, 1.0083596, 0.08954958, 0.7279827...",32500.0
4,12.0,"[-0.23105316, 1.0083596, 0.08954958, 0.7279827...",32500.0


In [36]:
dict_of_dfs = {key: val for key, val in df.groupby("cluster_label")}
dict_of_dfs.keys()

dict_keys([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0])

In [82]:
%load_ext autoreload
%autoreload 2

from core.models.clustering_model import StackedModels, hyperparameters_tuning

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
# params = {
#     "iterations": 10,
#     "learning_rate": 0.07060926866932606,
#     "depth": 10,
#     "subsample": 0.710251337503286,
#     "colsample_bylevel": 0.6883135875379539,
#     "min_data_in_leaf": 1,
# }
# 
# sample_model = StackedModels(**params)
# sample_model = sample_model.train(dict_of_dfs, test_size=0.2)

# # or

# params = hyperparameters_tuning(dict_of_dfs)


{}
0:	learn: 47577.6432816	total: 8.31ms	remaining: 74.8ms
1:	learn: 46589.0345788	total: 15.8ms	remaining: 63.1ms
2:	learn: 45559.7078350	total: 24.6ms	remaining: 57.5ms
3:	learn: 44517.1554869	total: 32.4ms	remaining: 48.6ms
4:	learn: 43550.4166412	total: 40.5ms	remaining: 40.5ms
5:	learn: 42595.0620505	total: 48.5ms	remaining: 32.3ms
6:	learn: 41820.4292491	total: 56ms	remaining: 24ms
7:	learn: 41008.1860314	total: 64.6ms	remaining: 16.1ms
8:	learn: 40267.0168294	total: 72.7ms	remaining: 8.08ms
9:	learn: 39596.0872044	total: 80.3ms	remaining: 0us
Model trained
{}
0:	learn: 10988.8826154	total: 609us	remaining: 5.49ms
1:	learn: 10612.6345103	total: 1.05ms	remaining: 4.21ms
2:	learn: 10301.9604146	total: 1.83ms	remaining: 4.27ms
3:	learn: 10034.5434144	total: 2.19ms	remaining: 3.29ms
4:	learn: 9787.2140010	total: 2.64ms	remaining: 2.64ms
5:	learn: 9510.3827502	total: 3.01ms	remaining: 2.01ms
6:	learn: 9244.1034293	total: 3.34ms	remaining: 1.43ms
7:	learn: 9000.7209697	total: 4.32ms	re

In [93]:
params = {
    "iterations": 10,
    "learning_rate": 0.07060926866932606,
    "depth": 10,
    "subsample": 0.710251337503286,
    "colsample_bylevel": 0.6883135875379539,
    "min_data_in_leaf": 1,
}

sample_model = StackedModels(**params)

split_dataset_dict = sample_model.split_dict_dataset(dict_of_dfs, test_size=0.2)
train_dataset_dict = {k: v for k, (v, _) in split_dataset_dict.items()}
test_dataset_dict = {k: v for k, (_, v) in split_dataset_dict.items()}

sample_model = sample_model.train(train_dataset_dict, test_size=0.0)

metric = sample_model.evaluate(dataset_dict=test_dataset_dict)
print(metric)

{}
0:	learn: 47577.6432816	total: 22.6ms	remaining: 204ms
1:	learn: 46589.0345788	total: 31ms	remaining: 124ms
2:	learn: 45559.7078350	total: 39.7ms	remaining: 92.6ms
3:	learn: 44517.1554869	total: 47.6ms	remaining: 71.5ms
4:	learn: 43550.4166412	total: 55.8ms	remaining: 55.8ms
5:	learn: 42595.0620505	total: 63.8ms	remaining: 42.5ms
6:	learn: 41820.4292491	total: 70.9ms	remaining: 30.4ms
7:	learn: 41008.1860314	total: 79.3ms	remaining: 19.8ms
8:	learn: 40267.0168294	total: 86.4ms	remaining: 9.6ms
9:	learn: 39596.0872044	total: 93.3ms	remaining: 0us
Model trained
{}
0:	learn: 10988.8826154	total: 563us	remaining: 5.07ms
1:	learn: 10612.6345103	total: 976us	remaining: 3.9ms
2:	learn: 10301.9604146	total: 1.64ms	remaining: 3.82ms
3:	learn: 10034.5434144	total: 2.07ms	remaining: 3.1ms
4:	learn: 9787.2140010	total: 2.53ms	remaining: 2.53ms
5:	learn: 9510.3827502	total: 2.92ms	remaining: 1.95ms
6:	learn: 9244.1034293	total: 3.27ms	remaining: 1.4ms
7:	learn: 9000.7209697	total: 4.33ms	remaini

In [94]:
print(sample_model.evaluate(test_dataset_dict))
sample_model.save_model("stacked_model.pkl")


{
    "0.0": 26031.076855614414,
    "1.0": 3391.46210675567,
    "2.0": 21199.02532730562,
    "3.0": 11933.120317095689,
    "4.0": 32961.47497209855,
    "5.0": 32200.171914251558,
    "6.0": 37756.44882186081,
    "7.0": 9479.600131278366,
    "8.0": 15129.827942679487,
    "9.0": 13278.413763894805,
    "10.0": 23777.910978376927,
    "11.0": 19151.38677169454,
    "12.0": 24726.813258664995,
    "13.0": 21438.627847467087,
    "14.0": 14612.137875745993,
    "15.0": 15733.502938466101,
    "16.0": 18006.504630665087
}
20047.500379642097
Model saved at stacked_model.pkl


In [95]:
sample_model = sample_model.load_model("stacked_model.pkl")
sample_model.evaluate(test_dataset_dict)

Model successfully loaded
{
    "0.0": 26031.076855614414,
    "1.0": 3391.46210675567,
    "2.0": 21199.02532730562,
    "3.0": 11933.120317095689,
    "4.0": 32961.47497209855,
    "5.0": 32200.171914251558,
    "6.0": 37756.44882186081,
    "7.0": 9479.600131278366,
    "8.0": 15129.827942679487,
    "9.0": 13278.413763894805,
    "10.0": 23777.910978376927,
    "11.0": 19151.38677169454,
    "12.0": 24726.813258664995,
    "13.0": 21438.627847467087,
    "14.0": 14612.137875745993,
    "15.0": 15733.502938466101,
    "16.0": 18006.504630665087
}


20047.500379642097